#MIP Analysis

This script analyses the data generated by the merger of the ownership data and the company panel information

In [1]:
import pandas as pd
import numpy as np

Load the datasets

In [2]:
df_merged_companies = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\outputs\merged_companies.csv")
df_merged_owners = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\outputs\merged_owners.csv")

C:\Users\lucas\AppData\Local\Temp\ipykernel_22172\2616293258.py:1: DtypeWarning: Columns (19,29,48,49,50,51,52,53,55,63,74,75,76,77,78,79,80,81,84,85,88,89,93,98,101,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,134,135,136,137,142,143,144,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,199,200,206,208,209,210,211,213,217,218,219,220,231,235,236,237,238,239,240,247,248,251,252,254,255,256,257,258,260,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,310,312,314,316,318,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,38

Two columns indicating if the company was involved in a ownership change in any of the panel years, one for `b_crefo` and one for `ownerid`

In [9]:
df_merged_companies["is_owner_change_company"] = df_merged_companies.groupby(["companyid"])["is_owner_change"].transform('any')
df_merged_companies["is_ownerid_change_company"] = df_merged_companies.groupby(["companyid"])["is_ownerid_change"].transform('any')

Further four columns were added, in order to indicate when the ownership change took place.

Change of ownership based on `b_crefo`: 
- 2009-2014 = `is_owner_change_2009`
- 2015-2017 = `is_owner_change_2015`

Change of ownership based on `ownerid`:
- 2009-2014 = `is_ownerid_change_2009`
- 2015-2017 = `is_ownerid_change_2015`

In [27]:
# b_crefo, 2009-2014
df_merged_companies["is_owner_change_2009"] = np.where((df_merged_companies["is_owner_change"] == True) & (df_merged_companies["panel_year"] > 2008) & (df_merged_companies["panel_year"] < 2015), True, False)
df_merged_companies["is_owner_change_2009"] = df_merged_companies.groupby(["companyid"])["is_owner_change_2009"].transform('any')

# b_crefo, 2015-2017
df_merged_companies["is_owner_change_2015"] = np.where((df_merged_companies["is_owner_change"] == True) & (df_merged_companies["panel_year"] > 2014) & (df_merged_companies["panel_year"] < 2018), True, False)
df_merged_companies["is_owner_change_2015"] = df_merged_companies.groupby(["companyid"])["is_owner_change_2015"].transform('any')

# ownerid, 2009-2014
df_merged_companies["is_ownerid_change_2009"] = np.where((df_merged_companies["is_ownerid_change"] == True) & (df_merged_companies["panel_year"] > 2008) & (df_merged_companies["panel_year"] < 2015), True, False)
df_merged_companies["is_ownerid_change_2009"] = df_merged_companies.groupby(["companyid"])["is_ownerid_change_2009"].transform('any')

# ownerid, 2015-2017
df_merged_companies["is_ownerid_change_2015"] = np.where((df_merged_companies["is_ownerid_change"] == True) & (df_merged_companies["panel_year"] > 2014) & (df_merged_companies["panel_year"] < 2018), True, False)
df_merged_companies["is_ownerid_change_2015"] = df_merged_companies.groupby(["companyid"])["is_ownerid_change_2009"].transform('any')

As previously discussed in the MIP merge file, the environmental innovations survey questions changed from 2009 to 2015 and 2021, and due to that, the individual variables do not all have the same meaning depending on the year (e.g. `oekpz6` in 2015 onwards refers to noise pollution, while in 2009 it refers to soil contamination). Therefore, I will construct the variables `oekpz_avg` and `oekpd_avg`, which are averages of the other variables and can show a holistic view of eco-innovation at the respective companies.

In [29]:
eco_innovations = ["oekpz1", "oekpz2", "oekpz3", "oekpz4", "oekpz5", "oekpz6", "oekpz7", "oekpz8", "oekpz9"] 
eco_product_innovations = ["oekpd1", "oekpd2", "oekpd3", "oekpd4"]
df_merged_companies["oekpz_avg"] = df_merged_companies[eco_innovations].mean(axis=1)
df_merged_companies["oekpd_avg"] = df_merged_companies[eco_product_innovations].mean(axis=1)
df_merged_owners["oekpz_avg"] = df_merged_owners[eco_innovations].mean(axis=1)
df_merged_owners["oekpd_avg"] = df_merged_owners[eco_product_innovations].mean(axis=1)

TypeError: unsupported operand type(s) for +: 'int' and 'str'